## Introduzione

Importiamo i moduli (librerie) necessarie per l'analisi:

- `numpy`: libreria per la gestione di array, matrici e funzioni numeriche.
- `scipy`: libreria di funzioni matematiche di alto livello per l'ottimizzazione, per l'algebra lineare, l'integrazione, funzioni speciali, ecc. In particolare, di `scipy` usiamo i moduli `optimize` e `signal`.
- `matplotlib`: libreria grafica di python per la produzione di plot e grafici.

In [ ]:
import numpy as np
from scipy import optimize, signal

import matplotlib.pyplot as plt
%matplotlib inline

from IPython.display import display, Math, Markdown

### Importare i dati

Tramite la funzione `loadtxt` di `numpy` è possibile importare dati da file. Nel nostro caso, importiamo i dati dal file `oscillazione.csv` che si trova nella sottocartella `data`.

Ispezionando il file, notiamo che i dati sono organizzati su **tre colonne**, delimitate da virgole. La prima riga del file contiene un'intestazione con i nomi delle colonne.
Assegniamo ciascuna colonna ad un `array` di `numpy`:

In [ ]:
theta,a,a_err = np.loadtxt('./data/oscillazione1.csv',delimiter = ',', skiprows = 1,unpack=True)

### Visualizzare i dati

È molto utile visualizzare i dati appena importati preparando un **plot** con `matplotlib`.<br>
Cominciamo a plottare le prime due colonne: l'angolo $\theta$ in **gradi** e l'ampiezza di oscillazione $a$, in **cm**.

In [ ]:
fig,ax = plt.subplots()

ax.set_xlabel(r'$\theta$ (°)',fontsize=12)
ax.set_ylabel('$a$ (cm)',fontsize=12)

ax.scatter(theta,a,c='b')

Per le elaborazioni conviene convertire l'angolo $\theta$ in radianti

In [ ]:
theta_rad = theta*np.pi/180.

In [ ]:
fig,ax = plt.subplots()

ax.set_xlabel(r'$\theta$ (rad)',fontsize=12)
ax.set_ylabel('$a$ (cm)',fontsize=12)

ax.scatter(theta_rad,a,c='b')

Possiamo migliorare il grafico cambiando i *tick* lungo l'asse *x*. Rappresentiamo solamente multipli di $\pi/2$ sui tick principali e di $\pi/4$ su quelli secondari

In [ ]:
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)

In [ ]:
tick_list = np.arange(0,7,1)*np.pi/2.
tick_labels = ['0','$\pi/2$','$\pi$','$3\pi/2$','$2\pi$','$5\pi/2$','$3\pi$']

In [ ]:
fig,ax = plt.subplots()

ax.set_xticks(tick_list)
ax.set_xticklabels(tick_labels)
ax.xaxis.set_minor_locator(MultipleLocator(np.pi/4.))
ax.grid(axis='x')

ax.set_xlabel(r'$\theta$ (rad)',fontsize=12)
ax.set_ylabel('$a$ (cm)',fontsize=12)

ax.scatter(theta_rad,a,c='b')

### Fit nonlineare

I dati sono evidentemente periodici con un periodo di circa $\pi$.<br>
Possiamo interpolare i dati con una funzione periodica del tipo
$$
f(\theta) = a_0 \cos b_0(\theta+\theta_0)
$$
che dipende da due parametri $a_0$, $b_0$ e $\theta_0$.

In [ ]:
def f(theta,a0,b0,theta0):
    return a0*np.cos(b0*(theta+theta0))

La funzione `curve_fit` di `scipy.optimize` usa il metodo dei **minimi quadrati** per ottenere il fit di dati con una qualsiasi funzione, anche non lineare.<br>
`curve_fit` richiede la funzione da fittare, definita prima, e i dati. <br>
In parentesi, possiamo specificare una stima dei parametri. In questo caso (30,0).

In [ ]:
params, covar = optimize.curve_fit(f,theta_rad,a)

Il primo array prodotto da `curve_fit`, chiamato `params` in questo caso, riporta i valori dei parametri `a0`, `b0` e `theta0` della funzione `f`.<br>
Possiamo subito valutare se l'ottimizzazione è andata a buon fine.

Valutiamo la funzione `f` con i parametri trovati su una lista di punti tra 0 e $3\pi$

In [ ]:
theta_ran = np.arange(0,3*np.pi,0.05)

In [ ]:
fig,ax = plt.subplots()

ax.set_xticks(tick_list)
ax.set_xticklabels(tick_labels)
ax.xaxis.set_minor_locator(MultipleLocator(np.pi/4.))
ax.grid(axis='x')

ax.set_xlabel(r'$\theta$ (rad)',fontsize=12)
ax.set_ylabel('$a$ (cm)',fontsize=12)

ax.scatter(theta_rad,a,c='b',label='Dati')

ax.plot(theta_ran,f(theta_ran,*params),c='r',label='Fit')

ax.legend(loc='best')

Evidentemente l'ottimizzazione non ha prodotto un risultato sensato. L'algoritmo ha bisogno di una stima ragionevole dei parametri. Nel nostro caso possiamo indicare, ad esempio

        a0 = 5.
        b0 = 2.
        theta0 = 0.

In [ ]:
params, covar = optimize.curve_fit(f,theta_rad,a,(5.,2.,0.))

In [ ]:
print(params)

In [ ]:
fig,ax = plt.subplots()

ax.set_xticks(tick_list)
ax.set_xticklabels(tick_labels)
ax.xaxis.set_minor_locator(MultipleLocator(np.pi/4.))
ax.grid(axis='x')

ax.set_xlabel(r'$\theta$ (rad)',fontsize=12)
ax.set_ylabel('$a$ (cm)',fontsize=12)

ax.scatter(theta_rad,a,c='b',label='Dati')

ax.plot(theta_ran,f(theta_ran,*params),c='r',label='Fit')

ax.legend(loc='best')

L'ottimizzazione è andata a buon fine. Possiamo valutare anche l'errore stimato sui parametri utilizzando il secondo risultato prodotto da `curve_fit`.

Il secondo array prodotto da `curve_fit` è la matrice delle covarianze dei parametri del fit. Gli elementi sulla diagonale sono le **varianze** dei parametri `a0` e `theta0`:

In [ ]:
print(np.diag(covar))

Gli errori sulla stima dei parametri sono quindi:

In [ ]:
sigma_params = np.sqrt(np.diag(covar))
print(sigma_params)

In [ ]:
display(Markdown(r'$a_0$ = {:.2f} $\pm$ {:.2f} cm'.format(params[0],sigma_params[0])))
display(Markdown(r'$b_0$ = {:.3f} $\pm$ {:.3f} rad'.format(params[1],sigma_params[1])))
display(Markdown(r'$\theta_0$ = {:.3f} $\pm$ {:.3f} rad'.format(params[2],sigma_params[2])))

### Bontà del fit: analisi dei residui

Una prima analisi della qualità del fit si può avere valutando i **residui**:
$$
res_i = a_i - f(\theta_i),
$$
e graficandoli.

In [ ]:
res = a - f(theta_rad,*params)

In [ ]:
fig,ax = plt.subplots()

ax.set_xticks(tick_list)
ax.set_xticklabels(tick_labels)
ax.xaxis.set_minor_locator(MultipleLocator(np.pi/4.))
ax.grid(axis='x')

ax.set_xlabel(r'$\theta$ (rad)',fontsize=12)
ax.set_ylabel('res. (cm)',fontsize=12)

ax.scatter(theta_rad,res,c='g')
ax.axhline(0,c='k',lw=1)

Per avere un confronto diretto con il fit e i dati, si possono allineare i due plot:

In [ ]:
fig,(ax1,ax2) = plt.subplots(2,1,sharex=True,gridspec_kw={'height_ratios':[2,1]})

## Plot dati e fit

ax1.set_xticks(tick_list)
ax1.set_xticklabels(tick_labels)
ax1.xaxis.set_minor_locator(MultipleLocator(np.pi/4.))
ax1.grid(axis='x')

#ax1.set_xlabel(r'$\theta$ (rad)',fontsize=12)
ax1.set_ylabel('$a$ (cm)',fontsize=12)

ax1.scatter(theta_rad,a,c='b',label='Dati')

ax1.plot(theta_ran,f(theta_ran,*params),c='r',label='Fit')

ax1.legend(loc='lower left')


## Plot residui

ax2.set_xticks(tick_list)
ax2.set_xticklabels(tick_labels)
ax2.xaxis.set_minor_locator(MultipleLocator(np.pi/4.))
ax2.grid(axis='x')

ax2.set_ylim(-0.6,0.6)

ax2.set_xlabel(r'$\theta$ (rad)',fontsize=12)
ax2.set_ylabel('res. (cm)',fontsize=12)

ax2.scatter(theta_rad,res,c='g')
ax2.axhline(0,c='k',lw=1)

La figura ottenuta, che è stata chiamata `fig` può essere salvata:

In [ ]:
fig.savefig('./fit_oscillazioni.png',dpi=200,bbox_inches='tight')

### Barre d'errore

La terza colonna del file di dati rappresenta l'errore sulla misura di $a$. Possiamo rappresentare anche questa grandezza usando le **barre d'errore** nel plot

In [ ]:
fig,ax = plt.subplots()

ax.set_xlabel(r'$\theta$ (rad)',fontsize=12)
ax.set_ylabel('$a$ (cm)',fontsize=12)

ax.errorbar(theta_rad,a,yerr=a_err,c='b',
            ls='none',marker='o',ms=2,capsize = 3, capthick = 1)

`curve_fit` può anche ottimizzare il fit tenendo conto dell'errore sui punti sperimentali. Gli errori su $a$, contenuti nell'array `a_err` sono passati a `curve_fit` specificando il parametro `sigma`

In [ ]:
params2, covar2 = optimize.curve_fit(f,theta_rad,a,(4.,2.,0.),sigma=a_err)

In [ ]:
print(params2)

In [ ]:
sigma_params2 = np.sqrt(np.diag(covar2))
print(sigma_params2)

In [ ]:
fig,ax = plt.subplots()

## Plot dati e fit

ax.set_xticks(tick_list)
ax.set_xticklabels(tick_labels)
ax.xaxis.set_minor_locator(MultipleLocator(np.pi/4.))
ax.grid(axis='x')

ax.set_xlabel(r'$\theta$ (rad)',fontsize=12)
ax.set_ylabel('$a$ (cm)',fontsize=12)

ax.errorbar(theta_rad,a,yerr=a_err,c='b',
            ls='none',marker='o',ms=2,capsize = 3, capthick = 1,label='Dati')

ax.plot(theta_ran,f(theta_ran,*params2),c='r',label='Fit')

ax.legend(loc='best')

Disponendo di un errore sui valori sperimentali, possiamo confrontarlo con i residui del fit

In [ ]:
res2 = a - f(theta_rad,*params2)

In [ ]:
fig,ax = plt.subplots()

ax.set_xticks(tick_list)
ax.set_xticklabels(tick_labels)
ax.xaxis.set_minor_locator(MultipleLocator(np.pi/4.))
ax.grid(axis='x')

ax.set_xlabel(r'$\theta$ (rad)',fontsize=12)
ax.set_ylabel('res (cm)',fontsize=12)

ax.errorbar(theta_rad,res2,yerr=2*a_err,c='r',
            ls='none',capsize = 3, capthick = 1,label='$\pm2\sigma$')
ax.errorbar(theta_rad,res2,yerr=a_err,c='b',
            ls='none',marker='o',ms=2,capsize = 3, capthick = 1,label='$\pm\sigma$')
ax.axhline(0,c='k',lw=1)

ax.legend(loc='best')

Un'altra può essere fatta rapportando ciascun residuo $res_i=a_i-f(\theta_i)$ al corrispondente errore $\sigma_i$ sul dato:
$$
\overline{res}_i=\frac{res_i}{\sigma_i}=\frac{a_i-f(\theta_i)}{\sigma_i}
$$

In [ ]:
res2_bar = res2/a_err

In [ ]:
fig,ax = plt.subplots()

ax.set_xticks(tick_list)
ax.set_xticklabels(tick_labels)
ax.xaxis.set_minor_locator(MultipleLocator(np.pi/4.))
ax.grid(axis='x')

ax.set_xlabel(r'$\theta$ (rad)',fontsize=12)
ax.set_ylabel('$\overline{res}$',fontsize=12)

ax.scatter(theta_rad,res2_bar,c='b')
ax.axhline(0,c='k',lw=1)

#ax.legend(loc='best')

Per avere tutti valori **positivi**, possiamo considerare
$$
\overline{res}_i^2=\left[\frac{a_i-f(\theta_i)}{\sigma_i}\right]^2
$$

In [ ]:
res2_bar_square = res2_bar**2

In [ ]:
fig,ax = plt.subplots()

ax.set_xticks(tick_list)
ax.set_xticklabels(tick_labels)
ax.xaxis.set_minor_locator(MultipleLocator(np.pi/4.))
ax.grid(axis='x')

ax.set_xlabel(r'$\theta$ (rad)',fontsize=12)
ax.set_ylabel('$\overline{res}^2$',fontsize=12)

ax.scatter(theta_rad,res2_bar_square,c='b')
ax.axhline(0,c='k',lw=1)

#ax.legend(loc='best')

## Oscillazioni smorzate

Seguendo lo schema dell'analisi precedente, provate ad ottenere il fit di un'**oscillazione smorzata**.<br>

La funzione da interpolare è del tipo
$$
g(t) = e^{-\gamma t}[A\cos(\omega t)+B\sin(\omega t)],
$$
e dipende da quattro parametri: $\gamma$, $\omega$, $A$ e $B$.


1) Importare il file `./data/oscillazioni_smorzate.csv` usando la funzione `np.loadtxt`.
2) Graficare i dati usando `matplotlib`.
3) Definire la funzione `g(t,gamma,w,A,B)`. Le funzioni di elementari da usare sono: `np.exp, np.sin, np.cos`
4) Interpolare i dati con la funzione `g` usando `optimize.curve_fit`.<br>
Si riesce ad ottenere il fit anche senza passare a `curve_fit` stime per i parametri? Quali stime possiamo fornire per aiutare l'algoritmo?
5) Quanto valgono i parametri $\gamma$ e $\omega$? Quali sono gli errori $\sigma_\gamma$ e $\sigma_\omega$?